In [ ]:
import pandas as pd
import geopandas as gpd
import random
from shapely.geometry import Point

In [2]:
#Medellin y AMVA

filename='data/001_EOD_2017_BD_General.xlsx'
df = pd.read_excel(filename)

In [3]:
df.head()

,ID_HOGAR,ID_MORADOR,ID_CONCAT,EDAD,ID_GENERO,DESC_GENERO,DESC_ESCOLARIDAD_MAX,DESC_OCUPACION,ESTRATO,SEC_VIAJE,...,M_Principal,DESC_M_Principal,FREC_CANTIDAD,ID_FREC_VIAJE,DESC_ID_FREC_VIAJE,COSTO_TTE_PARQ,FORMA_PAGO_PARQUEADERO,DESC_FORMA_PAGO_PARQUEADERO,TIPO_PARQUEO,DESC_TIPO_PARQUEO
0,1000067,396,1000067396,48,1,Masculino,Educación No formal,Trabajador Independiente,ESTRATO 3,624,...,1,Auto,1.0,4,Anual,2500.0,NaN,NaN,3,Lote o edificio pagado
1,1000067,396,1000067396,48,1,Masculino,Educación No formal,Trabajador Independiente,ESTRATO 3,625,...,1,Auto,1.0,4,Anual,NaN,NaN,NaN,5,Propio(7)
2,1000067,396,1000067396,48,1,Masculino,Educación No formal,Trabajador Independiente,ESTRATO 3,626,...,1,Auto,1.0,1,Diario,30000.0,NaN,NaN,3,Lote o edificio pagado
3,1000067,397,1000067397,32,2,Femenino,Técnico,Ama de casa,ESTRATO 3,627,...,10,Caminata,1.0,1,Diario,NaN,NaN,NaN,NaN,NaN
4,1000067,397,1000067397,32,2,Femenino,Técnico,Ama de casa,ESTRATO 3,628,...,10,Caminata,1.0,1,Diario,NaN,NaN,NaN,NaN,NaN


In [4]:

to_ndrop = ['MUNICIPIO_O','COMUNA_O','HORA_O',
            'MUNICIPIO_D','COMUNA_D','HORA_D',
            'DESC_MOTIVO_VIAJE','DESC_MODO_TTE_E1']
df=df[to_ndrop]

renaming={'COMUNA_O':'comuna_o',
        'MUNICIPIO_O':'municipio_o',
        'COMUNA_D':'comuna_d',
        'MUNICIPIO_D':'municipio_d',
        'HORA_O':'hora_o',
        'HORA_D':'hora_d',
        'DESC_MODO_TTE_E1':'modo_tte',
        'DESC_MOTIVO_VIAJE':'motivo_viaje'
        }

df=df.rename(columns=renaming)

df=df[df['modo_tte'].str.contains(r"Moto \(", regex=True)]

In [5]:
# Normalizar campos

normalize = {'Regreso a casa': 'Casa', 
           'Al Trabajo': 'Trabajo',
           'Recreaci√≥n':'Recreacion',
           'Diligencia o tr√°mite':'Diligencia',
           'Acompa√±ar a alguien':'Alguien',
           'Recoger o dejar a alguien':'Alguien'}

df['motivo_viaje'] = df['motivo_viaje'].replace(normalize)

In [6]:
#Generacion Medellin y AMVA

df_med=df[(df['municipio_o']=='Medellin') & (df['municipio_d']=='Medellin')].copy()

amva=['Medellin','Sabaneta','Bello','Caldas','Itagui','La Estrella',
      'Girardota','Envigado','Barbosa','Copacabana']
df_amva=df[(df['municipio_o'].isin(amva)) & df['municipio_d'].isin(amva)].copy()

In [13]:
#Importar geojson

gdf_med=gpd.read_file("geojson/medellin.geojson")
gdf_med=gdf_med[['name','geometry']].iloc[:21]

gdf_amva=gpd.read_file("geojson/amva.geojson");
gdf_amva=gdf_amva[['name','geometry']].iloc[0:9]

In [14]:
def get_random_point_in_polygon(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(p):
            return [p.x,p.y]

In [15]:
def assign_random_point(row, gdf, other_gdf=None, municipio_col='municipio_o', comuna_col='comuna_o'):
    municipio = row[municipio_col].strip().lower()
    comuna = row.get(comuna_col, None)
    comuna = comuna.strip().lower() if comuna else None

    # Medellin or Bogotá with comuna
    if municipio in ('medellin', 'bogotá') and comuna:
        match = gdf[gdf['name'].str.contains(comuna, case=False)]
        if not match.empty:
            polygon = match.iloc[0].geometry
            lon, lat = get_random_point_in_polygon(polygon)
            return pd.Series([lon, lat])

    # Other municipios
    elif other_gdf is not None and not other_gdf.empty:
        match = other_gdf[other_gdf['name'].str.contains(municipio, case=False)]
        if not match.empty:
            polygon = match.iloc[0].geometry
            lon, lat = get_random_point_in_polygon(polygon)
            return pd.Series([lon, lat])

    # No match found
    return pd.Series([None, None])


In [16]:
# Generar puntos de origen y destino para medellin

df_med[['o_long','o_lat']] = df_med.apply(
    assign_random_point, axis=1, args=(gdf_med, df_amva,'municipio_o','comuna_o')
)
df_med[['d_long','d_lat']] = df_med.apply(
    assign_random_point, axis=1, args=(gdf_med, df_amva,'municipio_d','comuna_d')
)

In [17]:
df_med

,municipio_o,comuna_o,hora_o,municipio_d,comuna_d,hora_d,motivo_viaje,modo_tte,o_long,o_lat,d_long,d_lat
28,Medellin,La Candelaria,20:00:00,Medellin,Corregimiento Altavista,21:00:00,Casa,Moto (Conductor),-75.569672,6.239086,-75.627785,6.244614
227,Medellin,Guayabal,14:00:00,Medellin,La Candelaria,15:00:00,Diligencia,Moto (Conductor),-75.585411,6.219663,-75.564959,6.252687
239,Medellin,Guayabal,09:00:00,Medellin,El Poblado,09:20:00,Estudio,Moto (Conductor),-75.577267,6.226652,-75.572040,6.219370
240,Medellin,El Poblado,16:00:00,Medellin,Guayabal,16:20:00,Casa,Moto (Conductor),-75.558960,6.201331,-75.585844,6.223384
434,Medellin,Corregimiento Altavista,08:00:00,Medellin,Corregimiento Altavista,08:05:00,Otro,Moto (Conductor),-75.634189,6.243387,-75.626981,6.223082
...,...,...,...,...,...,...,...,...,...,...,...,...
87260,Medellin,Corregimiento San Cristobal,17:30:00,Medellin,Corregimiento San Cristobal,17:40:00,Casa,Moto (Conductor),-75.598492,6.298566,-75.650552,6.273031
87265,Medellin,Corregimiento San Cristobal,06:30:00,Medellin,El Poblado,07:00:00,Trabajo,Moto (Acompa√±ante),-75.620117,6.283534,-75.561863,6.189834
87266,Medellin,El Poblado,17:30:00,Medellin,Corregimiento San Cristobal,18:00:00,Casa,Moto (Acompa√±ante),-75.564238,6.191712,-75.640924,6.289205
87269,Medellin,Corregimiento San Cristobal,06:30:00,Medellin,Corregimiento San Cristobal,07:00:00,Trabajo,Moto (Conductor),-75.614122,6.288800,-75.621308,6.266527


In [18]:
# Generar puntos de origen y destino para AMVA

df_amva[['o_long','o_lat']]=df_amva.apply(
    assign_random_point, axis=1, args=(gdf_med, gdf_amva,'municipio_o','comuna_o')
)

df_amva[['d_long','d_lat']]=df_amva.apply(
    assign_random_point, axis=1, args=(gdf_med, gdf_amva,'municipio_d','comuna_d')
)

In [19]:
# Exportar datos

# df_amva.to_json("results/amva.json",orient="records", indent=2)
df_med.to_json('results/med.json',orient="records", indent=2)

In [ ]:
#Bogota

filename='data/d.xlsx'
df=pd.read_excel(filename)

In [ ]:
# Eliminar columnas innecesarias, y normalizarlas

to_ndrop=['localidad_ori','nom_mun_ori','localidad_des',
          'nom_mun_des','hora_ini','hora_fin',
          'modo_principal_agrupado','motivo_viaje']

df=df[to_ndrop]

renaming={'localidad_ori':'comuna_o',
          'nom_mun_ori':'municipio_o',
          'localidad_des':'comuna_d',
          'nom_mun_des':'municipio_d',
          'hora_ini':'hora_o',
          'hora_fin':'hora_d',
          'modo_principal_agrupado':'modo_tte',
          'motivo_viaje':'motivo_viaje'}

df=df.rename(columns=renaming)

In [ ]:
# Filtrar para Bogotá y transporte en motos

df=df[(df['municipio_o']=='Bogotá') & (df['municipio_d']=='Bogotá')]
df=df[df['modo_tte']=='MOTO']

In [ ]:
# Normalzar datos en campos

normalize={' A trabajar':'Trabajo',
           ' A regresar al hogar':'Casa',
           ' A acompañar a alguien sin remuneración':'Alguien',
           ' A realizar algún trámite personal':'Diligencia',
           ' A Estudiar':'Estudio',
           ' A realizar compras':'Recreacion',
           ' A llevar y/o dejar algo':'Diligencia',
           ' Conduzco vehículo como forma de trabajo':'Trabajo',
           ' A asuntos médicos personales':'Salud',
           ' A visitar a alguien':'Alguien',
           ' A realizar actividades físicas y/o deportivas':'Recreacion',
           ' A realizar actividades recreativas y culturales':'Recreacion',
           ' A buscar trabajo':'Trabajo',
           ' A asistir actividad de tipo religioso y/o culto':'Recreacion',
           ' A acompañar a alguien con remuneración':'Trabajo'
           }
df['motivo_viaje']=df['motivo_viaje'].replace(normalize)

In [ ]:
df_bog=df.copy()
df_bog.head()

,comuna_o,municipio_o,comuna_d,municipio_d,hora_o,hora_d,modo_tte,motivo_viaje
31,Ciudad Bolivar,Bogotá,Bosa,Bogotá,08:00:00,08:45:00,MOTO,Trabajo
32,Bosa,Bogotá,Ciudad Bolivar,Bogotá,18:15:00,19:00:00,MOTO,Casa
80,Bosa,Bogotá,Kennedy,Bogotá,08:00:00,08:45:00,MOTO,Alguien
81,Kennedy,Bogotá,Bosa,Bogotá,09:52:12,10:37:12,MOTO,Casa
82,Bosa,Bogotá,Kennedy,Bogotá,08:00:00,08:45:00,MOTO,Trabajo


In [2]:
gdf_bog=gpd.read_file("geojson/bog_ofi.geojson")

In [ ]:
gdf_bog=gdf_bog[['LocNombre','geometry']].rename(columns={'LocNombre':'name'})

In [ ]:
# Generar puntos para Bogotá

df_bog[['o_long','o_lat']]=df_bog.apply(
    assign_random_point, axis = 1, args = (gdf_bog, gdf_bog, 'municipio_o','comuna_o')   
)

df_bog[['d_long','d_lat']]=df_bog.apply(
    assign_random_point, axis = 1, args = (gdf_bog, gdf_bog, 'municipio_d','comuna_d')   
)

In [ ]:
# df_bog.to_json('results/bog.json',orient="records",indent=2)

In [9]:
import json
with open("geojson/bog_ofi1.geojson","r") as f:
        bog = json.load(f);

In [7]:
del bog["type"]

In [ ]:
for feature in bog["features"]:
    feature["type"]="Feature"
    geometry = feature["geometry"]
    geometry.rena

{'properties': {'OBJECTID': 81,
  'LocNombre': 'ANTONIO NARIÃ‘O',
  'LocAAdmini': 'Acuerdo 117 de 2003',
  'LocArea': 4879543.3864294,
  'LocCodigo': '15',
  'SHAPE_Length': 0.108973020105648,
  'SHAPE_Area': 0.000397341362785474},
 'geometry': {'rings': [[[-74.1307544629999, 4.59335334700006],
    [-74.1291650249999, 4.59327354900006],
    [-74.1282584339999, 4.59327351800005],
    [-74.1275922519999, 4.59327349400007],
    [-74.1263517549999, 4.59325121800009],
    [-74.1250004929999, 4.59331785100005],
    [-74.1221429069999, 4.59356224600009],
    [-74.1209023959999, 4.59378447500006],
    [-74.1195068199999, 4.59411783500008],
    [-74.117978327, 4.59451787400008],
    [-74.1168707239999, 4.59480678900007],
    [-74.1156745069999, 4.59516238000009],
    [-74.1146333579999, 4.59538461100004],
    [-74.1134371379999, 4.59580688500006],
    [-74.1115763369999, 4.59660700600006],
    [-74.109095254, 4.59782942100009],
    [-74.1085675709999, 4.59794479400006],
    [-74.108215991, 4.59